## Action2：使用DeepFM对movielens进行评分预测

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, get_feature_names

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv', encoding = "ISO-8859-1")

In [ ]:
movie_ratings = pd.merge(ratings, movies, on='movieId')

In [ ]:
movie_ratings.head()

In [ ]:
sparse_features = ['userId', 'movieId', 'timestamp', 'title', 'genres']
target = ['rating']

In [ ]:
for feature in sparse_features:
    lbe = LabelEncoder()
    movie_ratings[feature] = lbe.fit_transform(movie_ratings[feature])

In [ ]:
movie_ratings.head()

In [ ]:
fixlen_feature_columns = [SparseFeat(feature, movie_ratings[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
train, test = train_test_split(movie_ratings, test_size = 0.2)
train_model_input = {name: train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass 
model = DeepFM(linear_feature_columns, dnn_feature_columns, task = 'regression')
model.compile('adam', 'mse', metrics=['mse'])
history = model.fit(train_model_input, train[target].values, batch_size=64, epochs = 10, verbose = True, validation_split=0.2)

In [ ]:
test_model_input

In [ ]:
pred_ans = model.predict(test_model_input, batch_size = 64)

In [ ]:
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print('test RMSE', rmse)